Copyright (c) 2017/2018, Johannes Schöneberg. All rights reserved.
The default copyright laws apply.

In [1]:
%matplotlib inline

import lvm_read
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time

import os
import datetime

import pandas as pd

#own modules / functions
import sys
pythonPackagePath = "/Users/johannesschoeneberg/git/JohSchoeneberg/Confleezers/confleezers_data_analysis/"
sys.path.append(pythonPackagePath+"/modules")
import InputProcessing as inproc
import ForceFileProcessing as ffp

In [2]:
def getSecondsRelativeToCreationOfForceFile(forceFileCreationTimestamp, timeString):
    #convert the time string into a datetime tuple
    date_time = timeString.split(" ")
    dateSplit = date_time[0].split('-')
    timeSplit = date_time[1].split(':')
    otherFileTimeStamp = datetime.datetime(int(dateSplit[0]), 
                         int(dateSplit[1]), 
                         int(dateSplit[2]), 
                         int(timeSplit[0]), 
                         int(timeSplit[1]), 
                         int(timeSplit[2].split('.')[0]), 
                        0)
    #subtract the seconds
    forceFileTimeCreatedSeconds = time.mktime(forceFileTimeCreated.timetuple())
    otherFileCreatedSeconds = time.mktime(otherFileTimeStamp.timetuple())
    return(otherFileCreatedSeconds-forceFileTimeCreatedSeconds)
    

In [3]:
path = '/Volumes/Samsung_T3b/science/confleezers/2018-08-09/v8/'

inputParameters = pd.read_csv(path+'_info.csv',names=['key','value'])

inputDataFolder = inproc.getInputParameter(inputParameters,"input_data_folder")


inputParameters[0:5]

,key,value
0,input_data_folder,./
1,output_data_folder,./
2,vesicleID_of_that_day,v8
3,date,2018-08-09
4,movie_originalFilename,v8b1001.nd2


In [4]:
# read the timestamp from the force file automatically
forceFile_raw_dataShareFilename = inproc.getInputParameter(inputParameters,"forceFile_raw_dataShareFilename")
infilename = path+forceFile_raw_dataShareFilename
print(infilename)
t,x,y,forceFileTimeCreated,xforceraw,yforceraw = ffp.readForceFile(infilename)

# f
forceFile_startTime = forceFileTimeCreated.strftime('%Y-%m-%d %H:%M:%S')
print("forceFile_startTime, ",forceFile_startTime)



/Volumes/Samsung_T3b/science/confleezers/2018-08-09/v8/forceFile_raw.lvm
forceFile_startTime,  2018-08-09 14:45:59


In [5]:
#put in _info.csv 
row = inputParameters.loc[inputParameters['key'] == 'forceFile_startTime']
inputParameters.loc[row.index,'value'] = forceFile_startTime
print(inputParameters.loc[inputParameters['key'] == 'forceFile_startTime'])


                    key                value
15  forceFile_startTime  2018-08-09 14:45:59


In [6]:
# read the timestamp from movie manually (fiji, info of file)
#timestampMovie = "TextInfoItem_99/12/201 = 7:35:33 PM"
#movieStartTime = "2017-09-12 19:35:33"
movie_startTimes = inproc.getInputParameter(inputParameters,"movie_startTime").split(';')

seconds_ffStart_to_movieStart_firstMovie = -1 
#Nikon Movie:
for movie_startTime in movie_startTimes:
    print("movie_startTime,",movie_startTime)
    seconds_ffStart_to_movieStart = getSecondsRelativeToCreationOfForceFile(forceFileTimeCreated,movie_startTime)
    if(seconds_ffStart_to_movieStart_firstMovie == -1):
        seconds_ffStart_to_movieStart_firstMovie = seconds_ffStart_to_movieStart
    #print("movie_startTime_difference_fftime_seconds,",seconds_ffStart_to_movieStart)
    
    #put in _info.csv 
    row = inputParameters.loc[inputParameters['key'] == 'movie_startTime_difference_fftime_seconds']
    inputParameters.loc[row.index,'value'] = seconds_ffStart_to_movieStart
    print(inputParameters.loc[inputParameters['key'] == 'movie_startTime_difference_fftime_seconds'])
    
    
    print()

movie_startTime, 2018-08-09 14:49:29
                                         key value
7  movie_startTime_difference_fftime_seconds   210



In [7]:


UVstart_seconds_fftime = int(inproc.getInputParameter(inputParameters,"UVstart_seconds_fftime"))

movie_startTime_difference_UVstart_seconds = seconds_ffStart_to_movieStart_firstMovie-UVstart_seconds_fftime

#print("movie_startTime_difference_UVstart_seconds,",movie_startTime_difference_UVstart_seconds)

#put in _info.csv 

row = inputParameters.loc[inputParameters['key'] == 'movie_startTime_difference_UVstart_seconds']
inputParameters.loc[row.index,'value'] = movie_startTime_difference_UVstart_seconds
print(inputParameters.loc[inputParameters['key'] == 'movie_startTime_difference_UVstart_seconds'])

row = inputParameters.loc[inputParameters['key'] == 'UVstart_seconds_movietime']
inputParameters.loc[row.index,'value'] = UVstart_seconds_fftime+movie_startTime_difference_UVstart_seconds
print(inputParameters.loc[inputParameters['key'] == 'UVstart_seconds_movietime'])


time_between_frames_seconds = float(inproc.getInputParameter(inputParameters,"time_between_frames_seconds"))
row = inputParameters.loc[inputParameters['key'] == 'UVstart_frames_movie']
inputParameters.loc[row.index,'value'] = int(1/time_between_frames_seconds*(UVstart_seconds_fftime+movie_startTime_difference_UVstart_seconds))
print(inputParameters.loc[inputParameters['key'] == 'UVstart_frames_movie'])



#---------


UVend_seconds_fftime = int(inproc.getInputParameter(inputParameters,"UVend_seconds_fftime"))

row = inputParameters.loc[inputParameters['key'] == 'UVend_seconds_movietime']
inputParameters.loc[row.index,'value'] = UVend_seconds_fftime+movie_startTime_difference_UVstart_seconds
print(inputParameters.loc[inputParameters['key'] == 'UVend_seconds_movietime'])


time_between_frames_seconds = float(inproc.getInputParameter(inputParameters,"time_between_frames_seconds"))
row = inputParameters.loc[inputParameters['key'] == 'UVend_frames_movie']
inputParameters.loc[row.index,'value'] = int(1/time_between_frames_seconds*(UVend_seconds_fftime+movie_startTime_difference_UVstart_seconds))
print(inputParameters.loc[inputParameters['key'] == 'UVend_frames_movie'])









                                          key value
8  movie_startTime_difference_UVstart_seconds   -75
                          key value
17  UVstart_seconds_movietime   210
                     key value
18  UVstart_frames_movie   630
                        key value
20  UVend_seconds_movietime   510
                   key value
21  UVend_frames_movie  1531


# save data in _info.csv

In [8]:
inputParameters.to_csv(path+'_info.csv',index=False,header=False)

# Other sometimes maybe useful functions

In [ ]:

def getLastModifiedTime(filename):
    imageTimeCreated = os.path.getmtime(filename)
    #print(imageTimeCreated)

    
    timestamp = imageTimeCreated
    value = datetime.datetime.fromtimestamp(timestamp)
    
    stringtime = value.strftime('%Y-%m-%d %H:%M:%S')
#    print(stringtime)
    
    return([imageTimeCreated,stringtime])
    

In [14]:
getLastModifiedTime("./input/vesicle_tube_UV_movie.nd2")

[1505270544.0, '2017-09-12 19:42:24']

In [15]:
def getLVMdatetime(lvmTimeString,lvmDateString):
    dateString = lvmDateString
    dateSplit=dateString.split('/')
    

    timeString = lvmTimeString
    timeSplit=timeString.split(':')

    #test = datetime.datetime(2017, 6, 26, 18, 21, 16, 533499)
    # this is the time the file was r
    test = datetime.datetime(int(dateSplit[0]), 
                         int(dateSplit[1]), 
                         int(dateSplit[2]), 
                         int(timeSplit[0]), 
                         int(timeSplit[1]), 
                         int(timeSplit[2].split('.')[0]), 
                        0)
    
    #print(test.strftime('%Y-%m-%d %H:%M:%S'))
    return test

In [16]:
import os
import platform

def creation_date(path_to_file):
    """
    Try to get the date that a file was created, falling back to when it was
    last modified if that isn't possible.
    See http://stackoverflow.com/a/39501288/1709587 for explanation.
    """
    if platform.system() == 'Windows':
        return os.path.getctime(path_to_file)
    else:
        stat = os.stat(path_to_file)
        try:
            return stat.st_birthtime
        except AttributeError:
            # We're probably on Linux. No easy way to get creation dates here,
            # so we'll settle for when its content was last modified.
            return stat.st_mtime

In [17]:
getLastModifiedTime("./input/vesicle_tube_UV_movie.nd2")

[1505270544.0, '2017-09-12 19:42:24']